In [1]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [78]:
from nltk.tokenize import sent_tokenize

In [84]:
def read_article(file_name):
    #read the file as text
    with open(file_name,"r") as f:
        filedata = f.read()
    
    article  = sent_tokenize(filedata)
    sentences = []
    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    #sentences.pop() 
    
    return sentences

In [85]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 

In [86]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

In [90]:
def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    
    # Step 1 - Read text and tokenize
    sentences =  read_article(file_name)
    #print("Sentences : ",sentences)
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    
    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)
    
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)
    
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

In [91]:
# let's begin
generate_summary( "cnn.txt", 2)

Kabul, Afghanistan (CNN)To the United States military, he was an ISIS-K facilitator they feared was involved in a plot to attack Kabul's international airport.
To his family and colleagues at a US nonprofit, 43-year-old Zamarai Ahmadi was an aid worker applying for a US visa to get his family out of Taliban-controlled Afghanistan.
In the two weeks since US drone operatives fired a Hellfire missile at a car in a residential Kabul compound, two vastly different narratives have emerged about the man who his family say died alongside nine relatives.
The Pentagon maintains at least one ISIS-K facilitator was killed in what Chairman of the Joint Chiefs Gen. Mark Milley called a "righteous strike" on the compound on August 29.
In a statement, US Central Command pointed to "significant secondary explosions" as evidence of a "substantial amount of explosive material" in the vehicle.
A US official with knowledge of the operation told CNN Thursday that operatives tracked the car for about eight h